In [20]:
%cd /home/bap/hana/Sentiment-Classification/sentiment-classification/sentiment_classification

/home/bap/hana/Sentiment-Classification/sentiment-classification/sentiment_classification


In [21]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

In [22]:
class CFG:
    # hyperparameters
    MAX_LEN = 256
    TRAIN_BATCH_SIZE = 32
    VALID_BATCH_SIZE = 32
    EPOCHS = 1
    LEARNING_RATE = 1e-05

In [23]:
test_df = pd.read_table('data/test.txt', delimiter = ';', header = None)
test_df.columns = ['comment', 'sentiment']
test_df = test_df.reset_index(drop=True)
test_df.head(5)

,comment,sentiment
0,im feeling rather rotten so im not very ambiti...,sadness
1,im updating my blog because i feel shitty,sadness
2,i never make her separate from me because i do...,sadness
3,i left with my bouquet of red and yellow tulip...,joy
4,i was feeling a little vain when i did this one,sadness


In [24]:
target_list = {'joy': [1, 0, 0, 0, 0, 0], 
               'sadness': [0, 1, 0, 0, 0, 0], 
               'anger': [0, 0, 1, 0, 0, 0], 
               'fear': [0, 0, 0, 1, 0, 0], 
               'love': [0, 0, 0, 0, 1, 0], 
               'surprise': [0, 0, 0, 0, 0, 1]}

In [25]:
target_index = {0: 'joy', 
                1: 'sadness', 
                2: 'anger', 
                3: 'fear', 
                4: 'love', 
                5: 'surprise'}

In [26]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [27]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 6)
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

In [28]:
model = BERTClass()
optimizer = torch.optim.Adam(params =  model.parameters())

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
def load_ckp(checkpoint_fpath, model):
    checkpoint = torch.load(checkpoint_fpath, map_location='cpu')
    model.load_state_dict(checkpoint['state_dict'])
    return model

In [30]:
model = load_ckp('models/best_model.pt', model)

In [14]:
device = 'cpu'
accuracy = 0
for idx, example in enumerate(list(test_df['comment'])):
    encodings = tokenizer.encode_plus(
        example,
        None,
        add_special_tokens=True,
        max_length=CFG.MAX_LEN,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    model.eval()
    with torch.no_grad():
        input_ids = encodings['input_ids'].to(device, dtype=torch.long)
        attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
        token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
        output = model(input_ids, attention_mask, token_type_ids)
        final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
        accuracy += target_index[np.argmax(final_output)] == list(test_df['sentiment'])[idx]
accuracy = accuracy / len(list(test_df['sentiment'])) * 100
accuracy
        #print(target_index[np.argmax(final_output)], list(test_df['sentiment'])[idx])

92.45

In [31]:
i = 3
encodings = tokenizer.encode_plus(
        list(test_df['comment'])[i],
        None,
        add_special_tokens=True,
        max_length=CFG.MAX_LEN,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
)
model.eval()
with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    print("Comment: ", list(test_df['comment'])[i])
    print("Predict: ", target_index[np.argmax(final_output)], ", Ground truth: ", list(test_df['sentiment'])[i])

Comment:  i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived
Predict:  joy , Ground truth:  joy
